In [1]:
import numpy as np
import skimage as sk
import skimage.io as skio
import skimage.transform

In [2]:
# name of the input file
imname = 'data/melons.tif'

# read in the image
im = skio.imread(imname)
height, width = im.shape

# convert to double (might want to do this later on to save memory)    
im = sk.img_as_float(im)

#removes the frayed edges of the image
def crop_edges(im, perc=0.05):
    a, b = perc, 1-perc
    new_im = im[int(a * height):int(b * height), int(a * width):int(b * width)]
    return new_im

def get_channels(im):
    height = np.floor(im.shape[0] / 3.0).astype(np.int)
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]
    return (r, g, b)

def ssd_score(image1, image2):
    return np.sum((image1-image2) ** 2)
r, g, b = get_channels(im)
r = crop_edges(r)
g = crop_edges(g)
b = crop_edges(b)
r = np.roll(r, 96, axis=0)
r = np.roll(r, 3, axis=1)
b = np.roll(b, 96, axis=0)
b = np.roll(b, 3, axis=1)
def align(image1, image2, pyramids=8):
    best_x, best_y = 0,0
    for p in reversed(range(pyramids)):
        print('in for loop ' + str(p))
        scale = 1/(2 ** (p))
        print('scaling...')
        image1 = sk.transform.rescale(r, scale, anti_aliasing=True)
        image2 = sk.transform.rescale(g, scale, anti_aliasing=True)
        if p+1 == pyramids:
            height, width = image1.shape
            range_x, range_y = [0, width], [0, height]        
        else:
            range_x, range_y = [(best_x-1) * 2, (best_x+1) * 2], [(best_y-1) * 2, (best_y+1) * 2]
        best = float('inf')
        print('finding best')
        for i in range(range_x[0], range_x[1]):
                for j in range(range_y[0], range_y[1]):
                    a = np.roll(image1, i, axis=0)
                    b = np.roll(a, j, axis=1)
                    score = ssd_score(b, image2)
                    if score < best:
                        best = score
                        best_x, best_y = i,j
        print('best for ' + str(p) + " is " + str(best_x) + ','+str(best_y))
    return (best_x, best_y)
im_out = np.dstack([r, g, b])
skio.imshow(im_out)
skio.show()


FileNotFoundError: [Errno 2] No such file or directory: 'data/melons.tif'

In [137]:
r = 96,3

inf